# This module is to connect to MongoDB and create flattened attributes, categories, hours from AZ_restaurant_data.json file to AZ_Restaurant_Final_Data.csv file 

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

**Connecting to MonogoDB**

In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.yelp

**Read the collection AZ_restaurant_data from MonogDB that contains all the details about the Open Restaurants in Arizona that were reviewed between 2013 and 2017 in JSON format.**

In [4]:
AZ_restaurant_data_df = pd.DataFrame(list(db.AZ_restaurant_data.find()))

In [5]:
AZ_restaurant_data_df.shape

(7367, 15)

We are going to work on the details of 7367 Restaurants in the state of Arizona for our analysis and research

In [6]:
AZ_restaurant_data_df.head(3)

,_id,address,attributes,business_id,categories,city,hours,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,5af0abe6639bf0c541d80c5b,719 E Thunderbird Rd,"{'RestaurantsTableService': False, 'GoodForMea...",rDMptJYWtnMhpQu_rRXHng,"[Fast Food, Burgers, Restaurants]",Phoenix,{},33.607070,-112.064382,McDonald's,,85022,10,1.0,AZ
1,5af0abe6639bf0c541d80c5d,"777 E Thunderbird Rd, Ste 107","{'RestaurantsTableService': True, 'GoodForMeal...",1WBkAuQg81kokZIPMpn9Zg,"[Burgers, Restaurants]",Phoenix,"{'Monday': '11:00-22:00', 'Tuesday': '11:00-22...",33.607310,-112.063404,Charr An American Burger Bar,,85022,232,3.0,AZ
2,5af0abe6639bf0c541d80c78,1635 E Camelback Rd,"{'RestaurantsTableService': False, 'GoodForMea...",iPa__LOhse-hobC2Xmp-Kw,"[Restaurants, Burgers, Fast Food]",Phoenix,"{'Monday': '5:00-23:00', 'Tuesday': '5:00-23:0...",33.508765,-112.046240,McDonald's,,85016,34,3.0,AZ


The sample display of the dataframe shows nested dictionary and lists inside few columns. Inorder to read the information inside the nested dictionary and lists for data analysis and processing, we need to flatten the information as individual columns.

In [7]:
AZ_restaurant_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7367 entries, 0 to 7366
Data columns (total 15 columns):
_id             7367 non-null object
address         7367 non-null object
attributes      7367 non-null object
business_id     7367 non-null object
categories      7367 non-null object
city            7367 non-null object
hours           7367 non-null object
latitude        7367 non-null float64
longitude       7367 non-null float64
name            7367 non-null object
neighborhood    7367 non-null object
postal_code     7367 non-null object
review_count    7367 non-null int64
stars           7367 non-null float64
state           7367 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 863.4+ KB


**Create subsetting of the Restaurant data with Attributes and categories**

In [8]:
AZ_restaurant_BusiAndAttr_df = AZ_restaurant_data_df[['business_id','attributes','categories','hours']]
AZ_restaurant_BusiAndAttr_df.head(3)

,business_id,attributes,categories,hours
0,rDMptJYWtnMhpQu_rRXHng,"{'RestaurantsTableService': False, 'GoodForMea...","[Fast Food, Burgers, Restaurants]",{}
1,1WBkAuQg81kokZIPMpn9Zg,"{'RestaurantsTableService': True, 'GoodForMeal...","[Burgers, Restaurants]","{'Monday': '11:00-22:00', 'Tuesday': '11:00-22..."
2,iPa__LOhse-hobC2Xmp-Kw,"{'RestaurantsTableService': False, 'GoodForMea...","[Restaurants, Burgers, Fast Food]","{'Monday': '5:00-23:00', 'Tuesday': '5:00-23:0..."


**The below code will flatten the Restaurant attributes, categories and hours into individual columns for each restaurant. The point to consider here is that this can increase the sparsity of the dataframe.**

In [9]:
i = 0
while i < len(AZ_restaurant_data_df):
    attr_key = list(list(AZ_restaurant_BusiAndAttr_df.loc[[i],'attributes'])[0].keys())
    catg_list = list(AZ_restaurant_BusiAndAttr_df.loc[[i],'categories'])[0]
    hour_key = list(list(AZ_restaurant_BusiAndAttr_df.loc[[i],'hours'])[0].keys())
    j = 0
#### Attributes #####
    while j < len(attr_key):
        attr_dictKey = attr_key[j]
#        print(attr_dictKey)
        attr_value = AZ_restaurant_BusiAndAttr_df.loc[[i],'attributes'][i][attr_dictKey]
#        print(attr_value)
#        print(type(attr_value))
        j += 1
        if(type(attr_value) is not dict):
            AZ_restaurant_BusiAndAttr_df.loc[[i],attr_dictKey] = attr_value
        else:
#            print("Dict: ")
#            print(list(attr_value.keys()))
            attr_value_key= list(attr_value.keys())
            l = 0
            while l < len(attr_value_key):
                attr_dictkey_key = attr_value_key[l]
#                print(attr_dictkey_key)
#                print(l)
#                print(list(attr_value.values())[l])
                attr_value_value = list(attr_value.values())[l]
                l += 1
                if(type(attr_value.values()) is not dict):
                    attr_dictkey_key = attr_dictKey + str('_') + attr_dictkey_key
                    AZ_restaurant_BusiAndAttr_df.loc[[i],attr_dictkey_key] = attr_value_value
#### Categories #####
    j = 0
    while j < len(catg_list):
#        print(j)
        catg_list_elem = catg_list[j]
#        print(catg_list_elem)
        catg_list_elem1 = str('category_') + catg_list_elem
#        print(catg_list_elem1)
        AZ_restaurant_BusiAndAttr_df.loc[[i],catg_list_elem1] = 1
        j += 1
#### hours ####
    j = 0
    while j < len(hour_key):
        hour_dictKey = hour_key[j]
#        print(hour_dictKey)
        hour_value = AZ_restaurant_BusiAndAttr_df.loc[[i],'hours'][i][hour_dictKey]
#        print(hour_value)
#        print(type(hour_value))
        j += 1
        if(type(hour_value) is not dict):
            AZ_restaurant_BusiAndAttr_df.loc[[i],'hours_'+hour_dictKey] = hour_value
            
    i += 1
    os.system('cls')
#    print (i,end ='\n')

C:\Users\Joy\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Joy\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
AZ_restaurant_BusiAndAttr_df.head(3)

,business_id,attributes,categories,hours,RestaurantsTableService,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,Alcohol,Caters,HasTV,RestaurantsGoodForGroups,NoiseLevel,WiFi,RestaurantsAttire,RestaurantsReservations,OutdoorSeating,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsDelivery,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsTakeOut,GoodForKids,DriveThru,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,category_Fast Food,category_Burgers,category_Restaurants,hours_Monday,hours_Tuesday,hours_Friday,hours_Wednesday,hours_Thursday,hours_Sunday,hours_Saturday,WheelchairAccessible,category_Pizza,BusinessAcceptsBitcoin,category_Marketing,category_Men's Clothing,category_Graphic Design,category_Women's Clothing,category_Screen Printing,category_Advertising,category_Shopping,category_Web Design,category_Fashion,category_Local Services,category_Screen Printing/T-Shirt Printing,category_Professional Services,Open24Hours,BYOBCorkage,BYOB,Corkage,DogsAllowed,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,category_Gluten-Free,category_Indian,category_Seafood,category_American (Traditional),category_Food,category_Delis,category_Sandwiches,HappyHour,GoodForDancing,category_Nightlife,category_Japanese,category_Music Venues,category_Ramen,category_Arts & Entertainment,category_Bars,category_Noodles,category_Vietnamese,category_Event Planning & Services,category_American (New),category_Salad,category_Caterers,category_Breakfast & Brunch,category_Greek,category_Vegetarian,category_Mediterranean,category_Mexican,category_Asian Fusion,category_Thai,category_Steakhouses,category_Sushi Bars,category_Bubble Tea,category_Tex-Mex,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,CoatCheck,Smoking,category_Wine Bars,category_Chinese,category_Chicken Shop,category_Cocktail Bars,category_Venues & Event Spaces,category_Buffets,category_Tacos,category_Virtual Reality Centers,category_Arcades,category_Vegan,category_Cafes,category_Italian,category_Bakeries,category_Soup,category_Sports Bars,category_Chicken Wings,category_Hot Dogs,category_Food Delivery Services,category_Diners,category_Ice Cream & Frozen Yogurt,category_Beer Bar,category_Pubs,category_Barbeque,category_Beer,category_Wine & Spirits,category_Desserts,category_Pakistani,category_Hawaiian,category_Cupcakes,category_Custom Cakes,category_Eatertainment,category_Middle Eastern,category_Festivals,category_Polish,category_Juice Bars & Smoothies,category_Musicians,category_Performing Arts,category_Food Trucks,ByAppointmentOnly,category_Breweries,category_Gastropubs,category_Coffee & Tea,category_Casinos,category_Ethiopian,category_Lounges,category_Tapas Bars,category_Chocolatiers & Shops,category_Specialty Food,category_Fish & Chips,category_Dim Sum,category_Arabian,category_Grocery,category_Convenience Stores,category_Falafel,category_Shopping Centers,category_Trinidadian,category_Caribbean,RestaurantsCounterService,category_Ethnic Food,category_Bagels,category_Beer Gardens,category_Halal,category_Fitness & Instruction,category_Dance Studios,category_Active Life,category_Southern,category_Golf,category_Dance Clubs,category_Adult Entertainment,category_Personal Chefs,category_Financial Services,category_Check Cashing/Pay-day Loans,category_Currency Exchange,category_Live/Raw Food,category_Wraps,category_Teppanyaki,category_Cheesesteaks,category_Meat Shops,category_Soul Food,category_French

In [11]:
AZ_restaurant_BusiAndAttr_df.shape

(7367, 524)

**Merge the flattened attributes and categories dataframe with the original dataframe to map the corresponding details of the restaurants.**

In [12]:
AZ_restaurant_BusiAndAttr_df1 = pd.merge(AZ_restaurant_data_df, AZ_restaurant_BusiAndAttr_df, on='business_id')

In [13]:
AZ_restaurant_BusiAndAttr_df1.shape

(7367, 538)

In [14]:
AZ_restaurant_BusiAndAttr_df1.head(3)

,_id,address,attributes_x,business_id,categories_x,city,hours_x,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,attributes_y,categories_y,hours_y,RestaurantsTableService,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,Alcohol,Caters,HasTV,RestaurantsGoodForGroups,NoiseLevel,WiFi,RestaurantsAttire,RestaurantsReservations,OutdoorSeating,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsDelivery,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsTakeOut,GoodForKids,DriveThru,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,category_Fast Food,category_Burgers,category_Restaurants,hours_Monday,hours_Tuesday,hours_Friday,hours_Wednesday,hours_Thursday,hours_Sunday,hours_Saturday,WheelchairAccessible,category_Pizza,BusinessAcceptsBitcoin,category_Marketing,category_Men's Clothing,category_Graphic Design,category_Women's Clothing,category_Screen Printing,category_Advertising,category_Shopping,category_Web Design,category_Fashion,category_Local Services,category_Screen Printing/T-Shirt Printing,category_Professional Services,Open24Hours,BYOBCorkage,BYOB,Corkage,DogsAllowed,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,category_Gluten-Free,category_Indian,category_Seafood,category_American (Traditional),category_Food,category_Delis,category_Sandwiches,HappyHour,GoodForDancing,category_Nightlife,category_Japanese,category_Music Venues,category_Ramen,category_Arts & Entertainment,category_Bars,category_Noodles,category_Vietnamese,category_Event Planning & Services,category_American (New),category_Salad,category_Caterers,category_Breakfast & Brunch,category_Greek,category_Vegetarian,category_Mediterranean,category_Mexican,category_Asian Fusion,category_Thai,category_Steakhouses,category_Sushi Bars,category_Bubble Tea,category_Tex-Mex,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,CoatCheck,Smoking,category_Wine Bars,category_Chinese,category_Chicken Shop,category_Cocktail Bars,category_Venues & Event Spaces,category_Buffets,category_Tacos,category_Virtual Reality Centers,category_Arcades,category_Vegan,category_Cafes,category_Italian,category_Bakeries,category_Soup,category_Sports Bars,category_Chicken Wings,category_Hot Dogs,category_Food Delivery Services,category_Diners,category_Ice Cream & Frozen Yogurt,category_Beer Bar,category_Pubs,category_Barbeque,category_Beer,category_Wine & Spirits,category_Desserts,category_Pakistani,category_Hawaiian,category_Cupcakes,category_Custom Cakes,category_Eatertainment,category_Middle Eastern,category_Festivals,category_Polish,category_Juice Bars & Smoothies,category_Musicians,category_Performing Arts,category_Food Trucks,ByAppointmentOnly,category_Breweries,category_Gastropubs,category_Coffee & Tea,category_Casinos,category_Ethiopian,category_Lounges,category_Tapas Bars,category_Chocolatiers & Shops,category_Specialty Food,category_Fish & Chips,category_Dim Sum,category_Arabian,category_Grocery,category_Convenience Stores,category_Falafel,category_Shopping Centers,category_Trinidadian,category_Caribbean,RestaurantsCounterService,category_Ethnic Food,category_Bagels,category_Beer Gardens,category_Halal,category_Fitness & Instruction,category_Dance Studios,category_Active Life,category_Southern,category_Golf,category_Dance Clubs,category_Adult Entertainment,category_Personal Chefs,category_Financial Services,category_Check Cashing/Pay-day Loans,category_Currency Exchange,cat

In [15]:
list(AZ_restaurant_BusiAndAttr_df1.columns)

['_id',
 'address',
 'attributes_x',
 'business_id',
 'categories_x',
 'city',
 'hours_x',
 'latitude',
 'longitude',
 'name',
 'neighborhood',
 'postal_code',
 'review_count',
 'stars',
 'state',
 'attributes_y',
 'categories_y',
 'hours_y',
 'RestaurantsTableService',
 'GoodForMeal_dessert',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'GoodForMeal_dinner',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'Alcohol',
 'Caters',
 'HasTV',
 'RestaurantsGoodForGroups',
 'NoiseLevel',
 'WiFi',
 'RestaurantsAttire',
 'RestaurantsReservations',
 'OutdoorSeating',
 'BusinessAcceptsCreditCards',
 'RestaurantsPriceRange2',
 'BikeParking',
 'RestaurantsDelivery',
 'Ambience_romantic',
 'Ambience_intimate',
 'Ambience_classy',
 'Ambience_hipster',
 'Ambience_divey',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'Ambience_casual',
 'RestaurantsTakeOut',
 'GoodForKids',
 'DriveThru',
 'BusinessParking_garage',
 'BusinessParking_street',
 'BusinessParking_validated',
 'Busine

### Below list of columns will be removed as they do not correspond to restaurant information.

In [16]:
Drop_col = \
['attributes_y',
'categories_y',
'hours_y',
'attributes_x',
'categories_x',
'hours_x',
'category_Marketing', 
"category_Men's Clothing", 
'category_Graphic Design', 
"category_Women's Clothing", 
'category_Screen Printing', 
'category_Advertising', 
'category_Shopping', 
'category_Web Design', 
'category_Fashion', 
'category_Local Services', 
'category_Screen Printing/T-Shirt Printing', 
'category_Professional Services', 
'category_Virtual Reality Centers', 
'category_Musicians', 
'category_Performing Arts', 
'category_Shopping Centers', 
'category_Fitness & Instruction', 
'category_Dance Studios', 
'category_Active Life', 
'category_Golf', 
'category_Dance Clubs', 
'category_Adult Entertainment', 
'category_Personal Chefs', 
'category_Financial Services', 
'category_Check Cashing/Pay-day Loans', 
'category_Currency Exchange', 
'category_Wholesale Stores', 
'category_Automotive', 
'category_Art Schools', 
'category_Internet Cafes', 
'category_Dance Schools', 
'category_Education', 
'category_Specialty Schools', 
'category_Preschools', 
'category_Butcher', 
'category_Poke', 
'category_Airport Lounges', 
'category_Shaved Ice', 
'category_Amusement Parks', 
'category_Mini Golf', 
'category_Bowling', 
'category_Acai Bowls', 
'category_Horseback Riding', 
'category_Karaoke', 
'category_Appliances', 
'category_Home & Garden', 
'category_Department Stores', 
'category_Bookstores', 
'category_Books', 
'category_Mags', 
'category_Music & Video', 
'category_Barbers', 
'category_Beauty & Spas', 
'category_Hair Salons', 
'category_Comedy Clubs', 
'category_Dive Bars', 
'category_Sporting Goods', 
'category_Massage Therapy', 
'category_Health & Medical', 
'category_Home Services', 
'category_Security Systems', 
'category_Playgrounds', 
'category_Fruits & Veggies', 
'category_Cinema', 
'category_Jazz & Blues', 
'category_Pharmacy', 
'category_Kids Activities', 
'category_Real Estate', 
'category_Country Clubs', 
'category_Sports Clubs', 
'category_Golf Equipment', 
'category_Wills', 
'category_Trusts', 
'category_& Probates', 
'category_Business Consulting', 
'category_Estate Planning Law', 
'category_Tax Services', 
'category_Bankruptcy Law', 
'category_Tax Law', 
'category_Lawyers', 
'category_Health Markets', 
'category_Cosmetics & Beauty Supply', 
'category_Gift Shops', 
'category_International Grocery', 
'category_Flowers & Gifts', 
'category_Ethnic Grocery', 
'category_Yoga', 
'category_Drugstores', 
'category_Tires', 
'category_Home Inspectors', 
'category_Interior Design', 
'category_Plumbing', 
'category_Damage Restoration', 
'category_Roofing', 
'category_Contractors', 
'category_Masonry/Concrete', 
'category_Flooring', 
'category_Country Dance Halls', 
'category_Community Service/Non-Profit', 
'category_Used Bookstore', 
'category_Libraries', 
'category_Public Services & Government', 
'category_Furniture Stores', 
'category_Grilling Equipment', 
'category_Kitchen & Bath', 
'category_Appliances & Repair', 
'category_Water Stores', 
'category_Car Wash', 
'category_Vitamins & Supplements', 
'category_Farmers Market', 
'category_Handyman', 
'category_Boating', 
'category_Real Estate Services', 
'category_Resorts', 
'category_Piano Bars', 
'category_Candy Stores', 
'category_Day Spas', 
'category_Florists', 
'category_Party Supplies', 
'category_Party Equipment Rentals', 
'category_Seafood Markets', 
'category_Pool Halls', 
'category_RV Parks', 
'category_Vacation Rentals', 
'category_Beverage Store', 
'category_Pool & Billiards', 
'category_Laser Tag', 
'category_Massage', 
'category_Medical Spas', 
'category_Club Crawl', 
'category_Horse Racing', 
'category_Stadiums & Arenas', 
'category_Swimming Pools', 
'category_Organic Stores', 
'category_Climbing', 
'category_Vape Shops', 
'category_Tobacco Shops', 
'category_Electronics', 
'category_Gyms', 
'category_Outlet Stores', 
'category_Golf Lessons', 
'category_Art Galleries', 
'category_Smokehouse', 
'category_Post Offices', 
'category_Museums', 
'category_Botanical Gardens', 
'category_Cultural Center', 
'category_Zoos', 
'category_Golf Equipment Shops', 
'category_Wedding Planning', 
'category_Pets', 
'category_Pet Services', 
'category_Pet Adoption', 
'category_Cheese Shops', 
'category_Social Clubs', 
'category_Cooking Classes', 
'category_Arts & Crafts', 
'category_Cooking Schools', 
'category_Hair Stylists', 
'category_Nail Technicians', 
'category_Nail Salons', 
'category_Dentists', 
'category_DJs', 
'category_Building Supplies', 
'category_Antiques', 
'category_Restaurant Supplies', 
'category_Wholesalers', 
'category_Shaved Snow', 
'category_Towing', 
'category_Whiskey Bars', 
'category_Accessories', 
'category_Shoe Stores', 
'category_Clothing Rental', 
'category_Lakes', 
'category_Wineries', 
'category_Home Cleaning', 
'category_Investing', 
'category_Property Management', 
'category_Doctors', 
'category_Pediatricians', 
'category_Cardiologists', 
'category_Hospitals', 
'category_Bike Repair/Maintenance', 
'category_Bikes', 
'category_Estate Liquidation', 
'category_Health Retreats', 
'category_Souvenir Shops', 
'category_Transportation', 
'category_Landmarks & Historical Buildings', 
'category_Foundation Repair', 
'category_Tours', 
'category_Magicians', 
'category_Supernatural Readings', 
'category_Life Coach', 
'category_Photographers', 
'category_Event Photography', 
'category_Session Photography', 
'category_Gay Bars', 
'category_Discount Store', 
'category_Air Duct Cleaning', 
'category_Fire Protection Services', 
'category_Office Cleaning', 
'category_Wedding Chapels', 
'category_Veterinarians', 
'category_Champagne Bars', 
'category_Auto Upholstery', 
'category_Furniture Reupholstery', 
'category_Auto Detailing', 
'category_Motorcycle Repair', 
'category_Auto Repair', 
'category_Auto Customization', 
'category_RV Repair', 
'category_Personal Shopping', 
'category_Couriers & Delivery Services', 
'category_Olive Oil', 
'category_Guest Houses', 
'category_Race Tracks', 
'category_Go Karts', 
'category_Leisure Centers', 
'category_Unofficial Yelp Events', 
'category_Christmas Trees', 
'category_Bingo Halls', 
'category_Trainers', 
'category_Nutritionists', 
'category_Shared Office Spaces', 
'category_Distilleries', 
'category_Farms', 
'category_Apartments', 
'category_Hot Tub & Pool', 
'category_Pool & Hot Tub Service', 
'category_Sports Wear', 
'category_Cosmetic Surgeons', 
'category_Jewelry', 
'category_Medical Transportation', 
'category_Orthodontists', 
'category_Teeth Whitening', 
'category_Home Health Care', 
'category_General Dentistry', 
'category_Cosmetic Dentists', 
'category_Rotisserie Chicken', 
'category_Airports', 
'category_Pawn Shops', 
'category_Tattoo', 
'category_Piercing',
'category_Venues & Event Spaces',
'category_Convenience Stores',
'category_Gas Stations',
'category_Hotels & Travel',
'category_Pita',
'category_Gelato',
'category_Party & Event Planning',
'category_Speakeasies',
'category_Empanadas',
'category_Bed & Breakfast',
'category_Food Tours',
'_id']

In [17]:
AZ_Restaurant_Final_Data = AZ_restaurant_BusiAndAttr_df1.drop(Drop_col,axis =1)

In [18]:
AZ_Restaurant_Final_Data.head(3)

,address,business_id,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,RestaurantsTableService,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,Alcohol,Caters,HasTV,RestaurantsGoodForGroups,NoiseLevel,WiFi,RestaurantsAttire,RestaurantsReservations,OutdoorSeating,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsDelivery,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsTakeOut,GoodForKids,DriveThru,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,category_Fast Food,category_Burgers,category_Restaurants,hours_Monday,hours_Tuesday,hours_Friday,hours_Wednesday,hours_Thursday,hours_Sunday,hours_Saturday,WheelchairAccessible,category_Pizza,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DogsAllowed,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,category_Gluten-Free,category_Indian,category_Seafood,category_American (Traditional),category_Food,category_Delis,category_Sandwiches,HappyHour,GoodForDancing,category_Nightlife,category_Japanese,category_Music Venues,category_Ramen,category_Arts & Entertainment,category_Bars,category_Noodles,category_Vietnamese,category_Event Planning & Services,category_American (New),category_Salad,category_Caterers,category_Breakfast & Brunch,category_Greek,category_Vegetarian,category_Mediterranean,category_Mexican,category_Asian Fusion,category_Thai,category_Steakhouses,category_Sushi Bars,category_Bubble Tea,category_Tex-Mex,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,CoatCheck,Smoking,category_Wine Bars,category_Chinese,category_Chicken Shop,category_Cocktail Bars,category_Buffets,category_Tacos,category_Arcades,category_Vegan,category_Cafes,category_Italian,category_Bakeries,category_Soup,category_Sports Bars,category_Chicken Wings,category_Hot Dogs,category_Food Delivery Services,category_Diners,category_Ice Cream & Frozen Yogurt,category_Beer Bar,category_Pubs,category_Barbeque,category_Beer,category_Wine & Spirits,category_Desserts,category_Pakistani,category_Hawaiian,category_Cupcakes,category_Custom Cakes,category_Eatertainment,category_Middle Eastern,category_Festivals,category_Polish,category_Juice Bars & Smoothies,category_Food Trucks,ByAppointmentOnly,category_Breweries,category_Gastropubs,category_Coffee & Tea,category_Casinos,category_Ethiopian,category_Lounges,category_Tapas Bars,category_Chocolatiers & Shops,category_Specialty Food,category_Fish & Chips,category_Dim Sum,category_Arabian,category_Grocery,category_Falafel,category_Trinidadian,category_Caribbean,RestaurantsCounterService,category_Ethnic Food,category_Bagels,category_Beer Gardens,category_Halal,category_Southern,category_Live/Raw Food,category_Wraps,category_Teppanyaki,category_Cheesesteaks,category_Meat Shops,category_Soul Food,category_French,category_Creperies,category_Street Vendors,category_Food Court,category_Donuts,category_Coffee Roasteries,category_Irish Pub,category_Hotels,category_Modern European,category_Korean,category_Pretzels,category_Cajun/Creole,category_Imported Food,category_Russian,AgesAllowed,category_Latin American,category_Colombian,category_Tea Rooms,category_Cantonese,category_Do-It-Yourself Food,category_Mongolian,category_Dominican,category_Local Flavor,category_Dinner Theater,category_Persian/Iranian,category_Szechuan,category_Comfort Food,AcceptsInsurance,category_Waffles,category_Brazilian,category_Herbs & Spices,category_Tapas/Small Plates,category_S

In [19]:
AZ_Restaurant_Final_Data.shape

(7367, 276)

In [20]:
list(AZ_Restaurant_Final_Data.columns)

['address',
 'business_id',
 'city',
 'latitude',
 'longitude',
 'name',
 'neighborhood',
 'postal_code',
 'review_count',
 'stars',
 'state',
 'RestaurantsTableService',
 'GoodForMeal_dessert',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'GoodForMeal_dinner',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'Alcohol',
 'Caters',
 'HasTV',
 'RestaurantsGoodForGroups',
 'NoiseLevel',
 'WiFi',
 'RestaurantsAttire',
 'RestaurantsReservations',
 'OutdoorSeating',
 'BusinessAcceptsCreditCards',
 'RestaurantsPriceRange2',
 'BikeParking',
 'RestaurantsDelivery',
 'Ambience_romantic',
 'Ambience_intimate',
 'Ambience_classy',
 'Ambience_hipster',
 'Ambience_divey',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'Ambience_casual',
 'RestaurantsTakeOut',
 'GoodForKids',
 'DriveThru',
 'BusinessParking_garage',
 'BusinessParking_street',
 'BusinessParking_validated',
 'BusinessParking_lot',
 'BusinessParking_valet',
 'category_Fast Food',
 'category_Burgers',
 'category_Res

In [21]:
col_list = ['business_id',
 'name',
 'neighborhood',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars',
 'review_count',
 'AcceptsInsurance',
 'AgesAllowed',
 'Alcohol',
 'Ambience_casual',
 'Ambience_classy',
 'Ambience_divey',
 'Ambience_hipster',
 'Ambience_intimate',
 'Ambience_romantic',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'BestNights_friday',
 'BestNights_monday',
 'BestNights_saturday',
 'BestNights_sunday',
 'BestNights_thursday',
 'BestNights_tuesday',
 'BestNights_wednesday',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking_garage',
 'BusinessParking_lot',
 'BusinessParking_street',
 'BusinessParking_valet',
 'BusinessParking_validated',
 'ByAppointmentOnly',
 'BYOB',
 'BYOBCorkage',
 'Caters',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions_dairy-free',
 'DietaryRestrictions_gluten-free',
 'DietaryRestrictions_halal',
 'DietaryRestrictions_kosher',
 'DietaryRestrictions_soy-free',
 'DietaryRestrictions_vegan',
 'DietaryRestrictions_vegetarian',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'GoodForKids',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'GoodForMeal_dessert',
 'GoodForMeal_dinner',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'HairSpecializesIn_africanamerican',
 'HairSpecializesIn_asian',
 'HairSpecializesIn_coloring',
 'HairSpecializesIn_curly',
 'HairSpecializesIn_extensions',
 'HairSpecializesIn_kids',
 'HairSpecializesIn_perms',
 'HairSpecializesIn_straightperms',
 'HappyHour',
 'HasTV',
 'Music_background_music',
 'Music_dj',
 'Music_jukebox',
 'Music_karaoke',
 'Music_live',
 'Music_no_music',
 'Music_video',
 'NoiseLevel',
 'Open24Hours',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsCounterService',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi',
 'category_Afghan',
 'category_African',
 'category_American (New)',
 'category_American (Traditional)',
 'category_Arabian',
 'category_Arcades',
 'category_Argentine',
 'category_Armenian',
 'category_Arts & Entertainment',
 'category_Asian Fusion',
 'category_Australian',
 'category_Austrian',
 'category_Bagels',
 'category_Bakeries',
 'category_Barbeque',
 'category_Bars',
 'category_Beer Bar',
 'category_Beer Gardens',
 'category_Beer',
 'category_Belgian',
 'category_Bistros',
 'category_Brasseries',
 'category_Brazilian',
 'category_Breakfast & Brunch',
 'category_Breweries',
 'category_Brewpubs',
 'category_British',
 'category_Bubble Tea',
 'category_Buffets',
 'category_Burgers',
 'category_Cafes',
 'category_Cafeteria',
 'category_Cajun/Creole',
 'category_Cambodian',
 'category_Cantonese',
 'category_Caribbean',
 'category_Casinos',
 'category_Caterers',
 'category_Cheesesteaks',
 'category_Chicken Shop',
 'category_Chicken Wings',
 'category_Chinese',
 'category_Chocolatiers & Shops',
 'category_Cocktail Bars',
 'category_Coffee & Tea Supplies',
 'category_Coffee & Tea',
 'category_Coffee Roasteries',
 'category_Coffeeshops',
 'category_Colombian',
 'category_Comfort Food',
 'category_Creperies',
 'category_Cuban',
 'category_Cupcakes',
 'category_Custom Cakes',
 'category_Czech',
 'category_Delis',
 'category_Desserts',
 'category_Dim Sum',
 'category_Diners',
 'category_Dinner Theater',
 'category_Do-It-Yourself Food',
 'category_Dominican',
 'category_Donuts',
 'category_Eatertainment',
 'category_Ethiopian',
 'category_Ethnic Food',
 'category_Event Planning & Services',
 'category_Falafel',
 'category_Fast Food',
 'category_Festivals',
 'category_Filipino',
 'category_Fish & Chips',
 'category_Fondue',
 'category_Food Court',
 'category_Food Delivery Services',
 'category_Food Stands',
 'category_Food Trucks',
 'category_Food',
 'category_French',
 'category_Gastropubs',
 'category_German',
 'category_Gluten-Free',
 'category_Greek',
 'category_Grocery',
 'category_Halal',
 'category_Hawaiian',
 'category_Herbs & Spices',
 'category_Hookah Bars',
 'category_Hot Dogs',
 'category_Hot Pot',
 'category_Hotel bar',
 'category_Hotels',
 'category_Ice Cream & Frozen Yogurt',
 'category_Imported Food',
 'category_Indian',
 'category_Indonesian',
 'category_Irish Pub',
 'category_Irish',
 'category_Italian',
 'category_Izakaya',
 'category_Japanese',
 'category_Juice Bars & Smoothies',
 'category_Kebab',
 'category_Korean',
 'category_Kosher',
 'category_Laotian',
 'category_Latin American',
 'category_Lebanese',
 'category_Live/Raw Food',
 'category_Local Flavor',
 'category_Lounges',
 'category_Macarons',
 'category_Malaysian',
 'category_Marinas',
 'category_Meat Shops',
 'category_Mediterranean',
 'category_Mexican',
 'category_Middle Eastern',
 'category_Modern European',
 'category_Mongolian',
 'category_Moroccan',
 'category_Music Venues',
 'category_New Mexican Cuisine',
 'category_Nightlife',
 'category_Noodles',
 'category_Pakistani',
 'category_Pan Asian',
 'category_Pasta Shops',
 'category_Patisserie/Cake Shop',
 'category_Persian/Iranian',
 'category_Peruvian',
 'category_Pizza',
 'category_Polish',
 'category_Popcorn Shops',
 'category_Poutineries',
 'category_Pretzels',
 'category_Pubs',
 'category_Puerto Rican',
 'category_Ramen',
 'category_Restaurants',
 'category_Russian',
 'category_Salad',
 'category_Salvadoran',
 'category_Sandwiches',
 'category_Scandinavian',
 'category_Seafood',
 'category_Soul Food',
 'category_Soup',
 'category_Southern',
 'category_Spanish',
 'category_Specialty Food',
 'category_Sports Bars',
 'category_Steakhouses',
 'category_Street Vendors',
 'category_Supper Clubs',
 'category_Sushi Bars',
 'category_Szechuan',
 'category_Tacos',
 'category_Taiwanese',
 'category_Tapas Bars',
 'category_Tapas/Small Plates',
 'category_Tea Rooms',
 'category_Teppanyaki',
 'category_Tex-Mex',
 'category_Thai',
 'category_Themed Cafes',
 'category_Trinidadian',
 'category_Turkish',
 'category_Ukrainian',
 'category_Uzbek',
 'category_Vegan',
 'category_Vegetarian',
 'category_Vietnamese',
 'category_Waffles',
 'category_Wine & Spirits',
 'category_Wine Bars',
 'category_Wraps',
 'hours_Monday',
 'hours_Tuesday',
 'hours_Wednesday',
 'hours_Thursday',
 'hours_Friday',
 'hours_Saturday',
 'hours_Sunday']

In [22]:
AZ_Restaurant_Final_Data = AZ_Restaurant_Final_Data[col_list]

In [23]:
AZ_Restaurant_Final_Data.head(3)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,AcceptsInsurance,AgesAllowed,Alcohol,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BestNights_friday,BestNights_monday,BestNights_saturday,BestNights_sunday,BestNights_thursday,BestNights_tuesday,BestNights_wednesday,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_lot,BusinessParking_street,BusinessParking_valet,BusinessParking_validated,ByAppointmentOnly,BYOB,BYOBCorkage,Caters,CoatCheck,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_halal,DietaryRestrictions_kosher,DietaryRestrictions_soy-free,DietaryRestrictions_vegan,DietaryRestrictions_vegetarian,DogsAllowed,DriveThru,GoodForDancing,GoodForKids,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_dessert,GoodForMeal_dinner,GoodForMeal_latenight,GoodForMeal_lunch,HairSpecializesIn_africanamerican,HairSpecializesIn_asian,HairSpecializesIn_coloring,HairSpecializesIn_curly,HairSpecializesIn_extensions,HairSpecializesIn_kids,HairSpecializesIn_perms,HairSpecializesIn_straightperms,HappyHour,HasTV,Music_background_music,Music_dj,Music_jukebox,Music_karaoke,Music_live,Music_no_music,Music_video,NoiseLevel,Open24Hours,OutdoorSeating,RestaurantsAttire,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,category_Afghan,category_African,category_American (New),category_American (Traditional),category_Arabian,category_Arcades,category_Argentine,category_Armenian,category_Arts & Entertainment,category_Asian Fusion,category_Australian,category_Austrian,category_Bagels,category_Bakeries,category_Barbeque,category_Bars,category_Beer Bar,category_Beer Gardens,category_Beer,category_Belgian,category_Bistros,category_Brasseries,category_Brazilian,category_Breakfast & Brunch,category_Breweries,category_Brewpubs,category_British,category_Bubble Tea,category_Buffets,category_Burgers,category_Cafes,category_Cafeteria,category_Cajun/Creole,category_Cambodian,category_Cantonese,category_Caribbean,category_Casinos,category_Caterers,category_Cheesesteaks,category_Chicken Shop,category_Chicken Wings,category_Chinese,category_Chocolatiers & Shops,category_Cocktail Bars,category_Coffee & Tea Supplies,category_Coffee & Tea,category_Coffee Roasteries,category_Coffeeshops,category_Colombian,category_Comfort Food,category_Creperies,category_Cuban,category_Cupcakes,category_Custom Cakes,category_Czech,category_Delis,category_Desserts,category_Dim Sum,category_Diners,category_Dinner Theater,category_Do-It-Yourself Food,category_Dominican,category_Donuts,category_Eatertainment,category_Ethiopian,category_Ethnic Food,category_Event Planning & Services,category_Falafel,category_Fast Food,category_Festivals,category_Filipino,category_Fish & Chips,category_Fondue,category_Food Court,category_Food Delivery Services,category_Food Stands,category_Food Trucks,category_Food,category_French,category_Gastropubs,category_German,category_Gluten-Free,category_Greek,category_Grocery,category_Halal,category_Hawaiian,category_Herbs & Spices,category_Hookah Bars,category_Hot Dogs,category_Hot Pot,category_Hotel bar,category_Hotels,category_Ice Cream & Frozen Yogurt,category_Imported Food,category_Indian,category_Indonesian,category_Irish Pub,category_Irish,category_Italian,category_Izakaya,category_Japanese,category_Juice Bars & Smoothies,category_Kebab,category_Korean,category_Kosher,category_Laotian,category_Latin American,category_Lebanese,category_Live/Raw Food,category_Local Flavor,category_Lounges,category_Macarons,category_Malaysian,category_Marinas,category_Meat Shops,category_Mediterranean,category_Mexican,category_Middle Eastern,category_Modern European,category_Mongolian,category

In [24]:
records = json.loads(AZ_Restaurant_Final_Data.T.to_json()).values()
db.AZ_Restaurant_Final_Data.insert_many(records)

In [25]:
AZ_Restaurant_Final_Data.to_csv("AZ_Restaurant_Final_Data.csv",sep=";")

### AZ_Restaurant_Final_Data.csv file will be used for further EDA processing in 2a_Restaurant_EDA_Cleansing.ipynb